In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

In [7]:
x_train = pd.read_csv('../data/train.csv', index_col=False)
x_test = pd.read_csv('../data/test.csv', index_col=False)

In [8]:
y_train = x_train.pop('label')
y_test = x_test.pop('label')

In [4]:
model = keras.models.Sequential()


In [12]:
capas = [
    keras.layers.Flatten(input_shape=(53, 53)),
    keras.layers.Dense(units = 300, activation='relu'),
    keras.layers.Dense(units = 100, activation='relu'),
    keras.layers.Dense(units = 2, activation='softmax')
]

model = keras.models.Sequential(capas)

In [7]:
model.compile(
    optimizer = "sgd",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [10]:
x_train.shape

(82332, 53)

In [11]:
y_train.shape

(82332,)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    batch_size = 128,
    epochs = 50,
    validation_split = 0.1
)

In [13]:
######################################### Neural Network ##################################################################

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from keras import metrics
# !pip install keras-metrics


In [2]:

import keras_metrics as km #when compiling
import keras
import numpy as np
from numpy import array

In [9]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [10]:
import time

In [33]:
#Build the feed forward neural network model
def build_model():
    model = Sequential()
    model.add(Dense(20, input_dim=53, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(20, activation='softmax')) #for multiclass classification
    #Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy',f1_m,precision_m, recall_m]
                 )
    return model

#institate the model
model = build_model()

#fit the model
#start = time.time()
model.fit(x_train, y_train, epochs=200, batch_size=2000,verbose=2)
#end_train = time.time()

Epoch 1/200
42/42 - 1s - loss: 2.1313 - accuracy: 0.5519 - f1_m: 0.0694 - precision_m: 0.6073 - recall_m: 0.0433 - 1s/epoch - 25ms/step
Epoch 2/200
42/42 - 0s - loss: 0.8387 - accuracy: 0.7238 - f1_m: 0.6898 - precision_m: 0.7009 - recall_m: 0.7238 - 219ms/epoch - 5ms/step
Epoch 3/200
42/42 - 0s - loss: 0.4547 - accuracy: 0.8125 - f1_m: 0.7074 - precision_m: 0.5602 - recall_m: 0.9609 - 189ms/epoch - 5ms/step
Epoch 4/200
42/42 - 0s - loss: 0.3561 - accuracy: 0.8415 - f1_m: 0.7086 - precision_m: 0.5519 - recall_m: 0.9899 - 183ms/epoch - 4ms/step
Epoch 5/200
42/42 - 0s - loss: 0.3018 - accuracy: 0.8584 - f1_m: 0.7101 - precision_m: 0.5519 - recall_m: 0.9957 - 203ms/epoch - 5ms/step
Epoch 6/200
42/42 - 0s - loss: 0.2672 - accuracy: 0.8722 - f1_m: 0.7111 - precision_m: 0.5524 - recall_m: 0.9981 - 177ms/epoch - 4ms/step
Epoch 7/200
42/42 - 0s - loss: 0.2437 - accuracy: 0.8796 - f1_m: 0.7098 - precision_m: 0.5506 - recall_m: 0.9986 - 215ms/epoch - 5ms/step
Epoch 8/200
42/42 - 0s - loss: 0.226

In [34]:
results = model.evaluate(x_test, y_test)
results

2573/2573 [==============================] - 4s 1ms/step - loss: 0.0916 - accuracy: 0.9626 - f1_m: 0.5507 - precision_m: 0.5506 - recall_m: 0.5511


[0.09162107110023499,
 0.9626269340515137,
 0.5507278442382812,
 0.5505732893943787,
 0.55110764503479]

In [35]:
from sklearn.metrics import plot_confusion_matrix

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# LSTM

In [30]:
def build_model():
    model = Sequential()
    model.add(LSTM(20, return_sequences=True,input_shape=(1,53)))
    model.add(LSTM(20, return_sequences=True))
    model.add(Dense(10, activation='softmax')) #for multiclass classification
    #Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
                  # metrics=['accuracy',f1_m,precision_m, recall_m]
                  metrics=['accuracy']
                 )
    return model

#The LSTM input layer must be 3D.
#The meaning of the 3 input dimensions are: samples, time steps, and features.
#reshape input data
x_train_array = array(x_train) #array has been declared in the previous cell
print(len(x_train_array))
x_train_reshaped = x_train_array.reshape(x_train_array.shape[0],1,53)

#reshape output data
x_test_array=  array(x_test)
x_test_reshaped = x_test_array.reshape(x_test_array.shape[0],1,53) 


#institate the model
model = build_model()


#fit the model
#start = time.time()
model.fit(x_train_reshaped, y_train, epochs=200, batch_size=2000,verbose=2)
#end_train = time.time()

82332
Epoch 1/200
42/42 - 4s - loss: 2.1981 - accuracy: 0.6017 - 4s/epoch - 89ms/step
Epoch 2/200
42/42 - 0s - loss: 1.6992 - accuracy: 0.7320 - 329ms/epoch - 8ms/step
Epoch 3/200
42/42 - 0s - loss: 0.8954 - accuracy: 0.7333 - 342ms/epoch - 8ms/step
Epoch 4/200
42/42 - 0s - loss: 0.5858 - accuracy: 0.7517 - 317ms/epoch - 8ms/step
Epoch 5/200
42/42 - 0s - loss: 0.4845 - accuracy: 0.7806 - 313ms/epoch - 7ms/step
Epoch 6/200
42/42 - 0s - loss: 0.4076 - accuracy: 0.8301 - 324ms/epoch - 8ms/step
Epoch 7/200
42/42 - 0s - loss: 0.3388 - accuracy: 0.8676 - 325ms/epoch - 8ms/step
Epoch 8/200
42/42 - 0s - loss: 0.2883 - accuracy: 0.8756 - 333ms/epoch - 8ms/step
Epoch 9/200
42/42 - 0s - loss: 0.2567 - accuracy: 0.8859 - 310ms/epoch - 7ms/step
Epoch 10/200
42/42 - 0s - loss: 0.2366 - accuracy: 0.8953 - 302ms/epoch - 7ms/step
Epoch 11/200
42/42 - 0s - loss: 0.2218 - accuracy: 0.9012 - 277ms/epoch - 7ms/step
Epoch 12/200
42/42 - 0s - loss: 0.2093 - accuracy: 0.9082 - 295ms/epoch - 7ms/step
Epoch 13/

In [32]:
results = model.evaluate(x_test, y_test)
results

ValueError: in user code:

    File "c:\Users\i2u3e\anaconda3\envs\deep\lib\site-packages\keras\engine\training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\i2u3e\anaconda3\envs\deep\lib\site-packages\keras\engine\training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\i2u3e\anaconda3\envs\deep\lib\site-packages\keras\engine\training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\i2u3e\anaconda3\envs\deep\lib\site-packages\keras\engine\training.py", line 1756, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\i2u3e\anaconda3\envs\deep\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\i2u3e\anaconda3\envs\deep\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compati